Importing all packages used.

In [1]:
import csv
import re
#import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
from nltk.tokenize import word_tokenize
#import contractions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import scipy as sp
from scipy.sparse import hstack
import random
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

random.seed(42)
np.random.seed(42)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\remya\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Reading the tsv into lists for text and class.

In [2]:
iFile = open('train.tsv', encoding="utf8")
iCSV = csv.reader(iFile, delimiter='\t')

X_text = []
y = []

for row in iCSV:
    X_text.append(row[1])
    y.append(row[2])
    
iFile.close()

iFile = open('test.tsv', encoding="utf8")
iCSV = csv.reader(iFile, delimiter='\t')

X_text_test = []
y_test = []

for row in iCSV:
    X_text_test.append(row[1])
    y_test.append(row[2])
    
iFile.close()

Viewing the data.

In [3]:
for tweet,Class in zip(X_text[:20],y[:20]):
    print(tweet,Class)
print()
print()
print()
for tweet,Class in zip(X_text[:75],y[:75]):
    if Class == 'TIN':
        print(tweet,Class)

@USER She should ask a few native Americans what their take on this is. UNT
@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL TIN
Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT NOT
@USER Someone should'veTaken" this piece of shit to a volcano. 😂" UNT
@USER @USER Obama wanted liberals &amp; illegals to move into red states NOT
@USER Liberals are all Kookoo !!! TIN
@USER @USER Oh noes! Tough shit. UNT
@USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism TIN
@USER Buy more icecream!!! NOT
@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo TIN
@USER @USER @USER It’s not my fault you support gun control NOT
@USER What’s the differe

Determining class balances.

In [4]:
UNT = 0 
TIN = 0
NOT = 0 
TOT = 0

for val in y:
    TOT += 1
    if val == 'UNT':
        UNT += 1
    elif val == 'TIN':
        TIN += 1
    else:
        NOT +=1 
        
        
print("UNT freq: {}, TIN freq: {}, NOT freq: {}".format(UNT/TOT,TIN/TOT,NOT/TOT))

UNT freq: 0.039746978851963745, TIN freq: 0.29286253776435045, NOT freq: 0.6673904833836858


Creating a train and validation split.

In [5]:
X_train_txt, X_val_txt, y_train_txt, y_val_txt = train_test_split(X_text, y, stratify = y)

Creating a boolean feature that records 1 if the tweet has an ! and a 0 if it does not.

In [6]:
ex_feature_train = []
ex_feature_val = []
ex_feature_test = []

for tweet in X_train_txt:
    if '!' in tweet:
        ex_feature_train.append([1])
    else:
        ex_feature_train.append([0])

for tweet in X_val_txt:
    if '!' in tweet:
        ex_feature_val.append([1])
    else:
        ex_feature_val.append([0])
        
for tweet in X_text_test:
    if '!' in tweet:
        ex_feature_test.append([1])
    else:
        ex_feature_test.append([0])
        
train_ex = sp.array(ex_feature_train)
val_ex = sp.array(ex_feature_val)
test_ex = sp.array(ex_feature_test)

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/4040373523.py:23: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  train_ex = sp.array(ex_feature_train)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/4040373523.py:24: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  val_ex = sp.array(ex_feature_val)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/4040373523.py:25: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  test_ex = sp.array(ex_feature_test)


Cleaning the data

Creating a dictionary to replace common contractions with the full words they represent.

In [7]:
contraction_dict = {

"ain’t": "is not",
 
"aren’t": "are not",

"can’t": "cannot",

"can’t’ve": "cannot have",

"’cause": "because",

"could’ve": "could have",

"couldn’t": "could not",

"couldn’t’ve": "could not have",

"didn’t": "did not",

"doesn’t": "does not",

"don’t": "do not",

"hadn’t": "had not",

"hadn’t’ve": "had not have",

"hasn’t": "has not",

"haven’t": "have not",

"he’d": "he would",

"he’d’ve": "he would have",

"he’ll": "he will",

"he’ll’ve": "he he will have",

"he’s": "he is",

"how’d": "how did",

"how’d’y": "how do you",

"how’ll": "how will",

"how’s": "how is",

"I’d": "I would",

"I’d’ve": "I would have",

"I’ll": "I will",

"I’ll’ve": "I will have",

"I’m": "I am",

"I’ve": "I have",

"i’d": "i would",

"i’d’ve": "i would have",

"i’ll": "i will",

"i’ll’ve": "i will have",

"i’m": "i am",

"i’ve": "i have",

"isn’t": "is not",

"it’d": "it would",

"it’d’ve": "it would have",

"it’ll": "it will",

"it’ll’ve": "it will have",

"it’s": "it is",

"let’s": "let us",

"ma’am": "madam",

"mayn’t": "may not",

"might’ve": "might have",

"mightn’t": "might not",

"mightn’t’ve": "might not have",

"must’ve": "must have",

"mustn’t": "must not",

"mustn’t’ve": "must not have",

"needn’t": "need not",

"needn’t’ve": "need not have",

"o’clock": "of the clock",

"oughtn’t": "ought not",

"oughtn’t’ve": "ought not have",

"shan’t": "shall not",

"sha’n’t": "shall not",

"shan’t’ve": "shall not have",

"she’d": "she would",

"she’d’ve": "she would have",

"she’ll": "she will",

"she’ll’ve": "she will have",

"she’s": "she is",

"should’ve": "should have",

"shouldn’t": "should not",

"shouldn’t’ve": "should not have",

"so’ve": "so have",

"so’s": "so as",

"that’d": "that would",

"that’d’ve": "that would have",

"that’s": "that is",

"there’d": "there would",

"there’d’ve": "there would have",

"there’s": "there is",

"they’d": "they would",

"they’d’ve": "they would have",

"they’ll": "they will",

"they’ll’ve": "they will have",

"they’re": "they are",

"they’ve": "they have",

"to’ve": "to have",

"wasn’t": "was not",

"we’d": "we would",

"we’d’ve": "we would have",

"we’ll": "we will",

"we’ll’ve": "we will have",

"we’re": "we are",

"we’ve": "we have",

"weren’t": "were not",

"what’ll": "what will",

"what’ll’ve": "what will have",

"what’re": "what are",

"what’s": "what is",

"what’ve": "what have",

"when’s": "when is",

"when’ve": "when have",

"where’d": "where did",

"where’s": "where is",

"where’ve": "where have",

"who’ll": "who will",

"who’ll’ve": "who will have",

"who’s": "who is",

"who’ve": "who have",

"why’s": "why is",

"why’ve": "why have",

"will’ve": "will have",

"won’t": "will not",

"won’t’ve": "will not have",

"would’ve": "would have",

"wouldn’t": "would not",

"wouldn’t’ve": "would not have",

"y’all": "you all",

"y’all’d": "you all would",

"y’all’d’ve": "you all would have",

"y’all’re": "you all are",

"y’all’ve": "you all have",

"you’d": "you would",

"you’d’ve": "you would have",

"you’ll": "you will",

"you’ll’ve": "you will have",

"you’re": "you are",

"you’ve": "you have"}

def replace_contraction(tweet):
    replaceList = []
    tweetList = tweet.split()
    for word in tweetList:
        if word in contraction_dict.keys():
            replaceList.append(contraction_dict[word])
        else:
            replaceList.append(word)
    return " ".join(replaceList)
X_train_pre_clean = []  
X_val_pre_clean = []
X_test_pre_clean = []
for tweet in X_train_txt:
    X_train_pre_clean.append(replace_contraction(tweet))
for tweet in X_val_txt:
    X_val_pre_clean.append(replace_contraction(tweet)) 
for tweet in X_text_test:
    X_test_pre_clean.append(replace_contraction(tweet))

In [8]:
def cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = re.sub('[.()!?]', '', tweet)
    tweet = re.sub('\[.*?\]','', tweet)

    return tweet
X_train_clean = []
X_val_clean = []
X_test_clean = []

for tweet in X_train_pre_clean:
    X_train_clean.append(cleaner(tweet))
    
for tweet in X_val_pre_clean:
    X_val_clean.append(cleaner(tweet))

for tweet in X_test_pre_clean:
    X_test_clean.append(cleaner(tweet))

Reading in an offensive word lexicon and creating a feature that counts the number of offensive words in each tweet.

In [9]:
offensive_words = []

iFile = open('Offensive words.txt')

for word in iFile:
    offensive_words.append(word.strip().lower())

iFile.close()
offensive_words = offensive_words[0:212]
offensive_words = set(offensive_words)

In [10]:
offensive_train_word_feature = []

for tweet in X_train_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in offensive_words:
            temp += 1
    offensive_train_word_feature.append([temp]) 
offensive_train_word_array = sp.array(offensive_train_word_feature)

offensive_val_word_feature = [] 

for tweet in X_val_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in offensive_words:
            temp += 1
    offensive_val_word_feature.append([temp]) 
offensive_val_word_array = sp.array(offensive_val_word_feature)

offensive_test_word_feature = []

for tweet in X_test_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in offensive_words:
            temp += 1
    offensive_test_word_feature.append([temp])
offensive_test_word_array = sp.array(offensive_test_word_feature)

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1377861510.py:9: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  offensive_train_word_array = sp.array(offensive_train_word_feature)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1377861510.py:19: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  offensive_val_word_array = sp.array(offensive_val_word_feature)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1377861510.py:29: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  offensive_test_word_array = sp.array(offensive_test_word_feature)


Reading in a positive word lexicon and creating a feature that counts the number of positive words in each tweet.

In [11]:
positive_words = []

iFile = open('positive-words.txt')

for word in iFile:
    positive_words.append(word.strip().lower())

iFile.close()
positive_words = set(positive_words)

In [12]:
positive_train_word_list = []

for tweet in X_train_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in positive_words:
            temp += 1
    positive_train_word_list.append([temp]) 
    
positive_train_word_feature = sp.array(positive_train_word_list)

positive_val_word_list = [] 

for tweet in X_val_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in positive_words:
            temp += 1
    positive_val_word_list.append([temp]) 
positive_val_word_feature = sp.array(positive_val_word_list)

positive_word_test_list = []

for tweet in X_test_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in positive_words:
            temp += 1 
    positive_word_test_list.append([temp])
positive_word_test_feature = sp.array(positive_word_test_list)

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/344155394.py:10: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  positive_train_word_feature = sp.array(positive_train_word_list)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/344155394.py:20: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  positive_val_word_feature = sp.array(positive_val_word_list)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/344155394.py:30: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  positive_word_test_feature = sp.array(positive_word_test_list)


Reading in a lexicon and creating a feature that counts words we found were highly associated with TIN in each tweet.

In [13]:
tin_wordsList = []
iFile = open('TINGroups.txt')

for word in iFile:
    tin_wordsList.append(word.strip().lower())

iFile.close()
tin_words = set(tin_wordsList)

In [14]:
train_tin_words = []

for tweet in X_train_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in tin_words:
            temp += 1
    train_tin_words.append([temp]) 
tin_word_train_feature = sp.array(train_tin_words)

val_tin_words = [] 

for tweet in X_val_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in tin_words:
            temp += 1
    val_tin_words.append([temp]) 
tin_word_val_feature = sp.array(val_tin_words)

tin_words_test_list = [] 

for tweet in X_test_clean:
    temp = 0
    for word in tweet.split():
        if word.lower() in tin_words:
            temp += 1
    tin_words_test_list.append([temp]) 
tin_words_test_feature = sp.array(tin_words_test_list)

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/2245093005.py:9: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  tin_word_train_feature = sp.array(train_tin_words)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/2245093005.py:19: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  tin_word_val_feature = sp.array(val_tin_words)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/2245093005.py:29: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  tin_words_test_feature = sp.array(tin_words_test_list)


We noticed that a proportion of the tweets containing "Trump" are targeted insults. Creating a vector containing the number of occurrences of the word 'trump' in each tweet.

In [15]:
Train_trump_count = []
Val_trump_count = []
trump_count_test_list = []

for tweet in X_train_clean:
    count = 0
    for word in tweet.lower().split():
        if word == 'trump':
            count += 1
    Train_trump_count.append([count])
    
for tweet in X_val_clean:
    count = 0
    for word in tweet.lower().split():
        if word == 'trump':
            count += 1
    Val_trump_count.append([count])
    
for tweet in X_test_clean:
    count = 0
    for word in tweet.lower().split():
        if word == 'trump':
            count += 1
    trump_count_test_list.append([count])

Trump_count_feature_train = sp.array(Train_trump_count)
Trump_count_feature_val = sp.array(Val_trump_count)
Trump_count_test_feature = sp.array(trump_count_test_list)

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/3106246390.py:26: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  Trump_count_feature_train = sp.array(Train_trump_count)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/3106246390.py:27: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  Trump_count_feature_val = sp.array(Val_trump_count)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/3106246390.py:28: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  Trump_count_test_feature = sp.array(trump_count_test_list)


Using the nltk library to count the number of personal pronouns in a tweet. The idea is that tweets with proper nouns are more likely to be targeted insults than tweets without proper nouns.

In [28]:
#nltk.tag.pos_tag(tokens, tagset=None, lang='eng')

from nltk.tag import pos_tag
#nltk.download('punkt')

tokenizedList = [word_tokenize(tweet) for tweet in X_train_clean]
tagListofLists = [nltk.pos_tag(List) for List in tokenizedList]

PropNounCountList = []

for List in tagListofLists:
    CountVariable = 0
    for Tuple in List:
        if Tuple[1] == "PRP":
            CountVariable += 1
    PropNounCountList.append([CountVariable])
PRPTrainFeat = sp.array(PropNounCountList)

tokenizedListVal = [word_tokenize(tweet) for tweet in X_val_clean]
tagListofListsVal = [nltk.pos_tag(List) for List in tokenizedListVal]

PropNounCountListVal = []

for List in tagListofListsVal:
    CountVariable = 0
    for Tuple in List:
        if Tuple[1] == "PRP":
            CountVariable += 1
    PropNounCountListVal.append([CountVariable])
PRPValFeat = sp.array(PropNounCountListVal)

tokenizedListVal = [word_tokenize(tweet) for tweet in X_test_clean]
tagListofListsVal = [nltk.pos_tag(List) for List in tokenizedListVal]

PropNounCountListTest = []

for List in tagListofListsVal:
    CountVariable = 0
    for Tuple in List:
        if Tuple[1] == "PRP":
            CountVariable += 1
    PropNounCountListTest.append([CountVariable])
PRPTestFeat = sp.array(PropNounCountListVal)

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\remya/nltk_data'
    - 'C:\\Users\\remya\\anaconda3\\nltk_data'
    - 'C:\\Users\\remya\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\remya\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\remya\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Converting to numpy arrays and creating bag of words matrix. Then experimenting with our different features.

In [18]:
r = TfidfVectorizer(ngram_range=(1,2),max_df = 0.5)
X_train_clean = sp.array(X_train_clean)
X_val_clean = sp.array(X_val_clean)
y_train = sp.array(y_train_txt)
y_val = sp.array(y_val_txt)
X_test_clean = sp.array(X_test_clean)
y_test = sp.array(y_test)

X_train = r.fit_transform(X_train_clean)
X_val = r.transform(X_val_clean)
X_test = r.transform(X_test_clean)

X_train = hstack([X_train, tin_word_train_feature, offensive_train_word_array, PRPTrainFeat])

X_val = hstack([X_val, tin_word_val_feature, offensive_val_word_array, PRPValFeat])

X_test = hstack([X_test, tin_words_test_feature, offensive_test_word_array, PRPTestFeat])

C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1477533737.py:2: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  X_train_clean = sp.array(X_train_clean)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1477533737.py:3: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  X_val_clean = sp.array(X_val_clean)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1477533737.py:4: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  y_train = sp.array(y_train_txt)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1477533737.py:5: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  y_val = sp.array(y_val_txt)
C:\Users\remya\AppData\Local\Temp/ipykernel_13452/1477533737.py:6: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array

NameError: name 'PRPTrainFeat' is not defined

In [114]:
#X_train = hstack([X_train,
                  offensive_train_word_array,
                  train_ex,
                  Trump_count_feature_train,
                  PRPTrainFeat,
                  positive_train_word_feature,
                  tin_word_train_feature])

#X_val = hstack([X_val, 
                offensive_val_word_array,
                val_ex,
                Trump_count_feature_val,
                PRPValFeat,
                positive_val_word_feature,
                tin_word_val_feature])

Fitting a linear SVC and using GridSearchCV to find the optimal parameters.

In [29]:
svc = LinearSVC()
params = {'C':[.1,1,10]}
clf = GridSearchCV(svc, params, scoring="f1_micro",cv=3)
clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=LinearSVC(), param_grid={'C': [0.1, 1, 10]},
             scoring='f1_micro')

Testing a Random Forest Classifier. This did not work as well as the LinearSVC.

In [30]:
rf = RandomForestClassifier()
params = {'n_estimators':[100]}
clf = GridSearchCV(rf, params, scoring="f1_micro",cv=3)
clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100]}, scoring='f1_micro')

Viewing results: Top f1 micro and f1 macro scores on training data.

In [31]:
print(clf.best_params_,clf.best_score_)
preds = clf.predict(X_train)
np.mean(cross_val_score(svc,X_train,y_train,scoring="f1_macro",cv=3))

{'n_estimators': 100} 0.7072004028197382


0.41377608562896845

Testing model using only bag of words for prediction. Note that these cells are only meant to be run when the model is fit with the corresponding features.

In [32]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
print(micro,macro)
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

0.7224320241691843 0.4321556060686495
F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Testing model using bag of words plus feature made from offensive word lexicon.

In [33]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Testing model using bag of words plus exclamation feature.

In [34]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Testing model using bag of words plus Trump word count feature.

In [35]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Testing bag of words with personal pronoun feature. 

In [36]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Bag of words with positive word feature.

In [37]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Bag of words with TIN feature.

In [38]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


All features

In [39]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Bag of words plus offensive word feature, targeted insult word feature and personal pronoun feature. 

In [40]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Random Forest Classifier

In [41]:
val_preds = clf.predict(X_val)
micro = f1_score(y_val, val_preds, average='micro')
macro = f1_score(y_val, val_preds, average='macro')
precision = precision_score(y_val, val_preds, average = "macro") # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_val, val_preds, average = "macro")

print("F1 Micro: {:.4f}".format(micro))
print("F1 Macro: {:.4f}".format(macro))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))

F1 Micro: 0.7224
F1 Macro: 0.4322
Precision: 0.6731
Recall: 0.4243


Conducting error analysis.

In [42]:
predictions = zip(list(val_preds),list(y_val))

untCorrect = []
untIncorrect = []

for i in range(len(val_preds)):
    if y_val[i] == "UNT":
        if val_preds[i] == "UNT":
            untCorrect.append([X_val_txt[i]])
        else:
            untIncorrect.append([X_val_txt[i],val_preds[i]])
            
untIncorrect[:50]    

[['@USER @USER @USER @USER @USER Oh and how do think the victim feels? Im sure she is fine with the death threats and insults she’s been getting spewed at her by you fine Christian conservatives',
  'NOT'],
 ['@USER we bout to start stepping out lol I ain’t doing another winter with no boo bro. Shit cold .... literally',
  'TIN'],
 ['@USER  Good- that soundtrack you DMed me with was fire as fuck and I still listen to it often💪🏽 Bad- My tracks are better😭 (joking I’m shit)',
  'TIN'],
 ['@USER Oh shit.', 'TIN'],
 ['@USER sex games will make you feel like you\'re actully having sex or, as ign would describe it it really makes you feel like you\'re fucking some whore in an alleyway""',
  'NOT'],
 ['@USER A vulgar attempt to reduce sisterhood to genitalia.....', 'NOT'],
 ['Opportunist to the max. An evil lier also. URL', 'NOT'],
 ["@USER Stop saying this shit you are not going to do it shrio you have to fight this depression you can't let it take you half of my life was me hiding my emotio

Reading in the test data to make predictions

In [43]:
testPreds = clf.predict(X_test)

testDict = {}
for val in testPreds:
    testDict[val] = testDict.get(val,0) + 1
    
print("test Dict", testDict)

valDict = {}
for val in val_preds:
    valDict[val] = valDict.get(val,0) + 1

print("val dict", valDict)

valTruthDict = {}
for val in y_val:
    valTruthDict[val] = valTruthDict.get(val,0) + 1

print("grnd tru",valTruthDict)

9/(2064+570+9)

test Dict {'NOT': 2418, 'TIN': 223, 'UNT': 7}
val dict {'NOT': 2419, 'TIN': 214, 'UNT': 15}
grnd tru {'NOT': 1767, 'TIN': 776, 'UNT': 105}


0.00340522133938706

Writing our test predictions to a text file.

In [44]:
oFile = open("predictions.tsv",'w', encoding="utf8")

tsvObj = csv.writer(oFile,delimiter='\t')
listForWriting = []
for i in range(len(X_text_test)):
    listForWriting.append([X_text_test[i],testPreds[i]])

tsvObj.writerows(listForWriting)

oFile.close()